数据预处理

In [1]:
import pandas as pd
import xlrd
xlsfile = "B_vol.xlsx"
book = xlrd.open_workbook(xlsfile)
table = book.sheet_by_index(1)
nrows = table.nrows
ncols = table.ncols
print("The table all row is %d,col is %s" %(nrows,ncols))

The table all row is 2128,col is 1


In [2]:
data = []
for i in range(nrows):
    data.append(table.cell(i,0).value)

In [3]:
data_rev = list(reversed(data))

处理掉断行的题目

In [4]:
import re
m = re.compile('(\d+\u3001)|([A-F]\.)')
f = [re.match(m,data_rev[i]) for i in range(len(data_rev))]

for i in range(len(f)-1):
    if f[i] is None:
        data_rev[i+1] = '%s%s' % (data_rev[i+1], data_rev[i])
        data_rev[i] = 'flag'    # flag是将要删掉的行的标记
        
for i in range(len(data_rev)-1,-1,-1):
    if data_rev[i] == 'flag':
        data_rev.pop(i)


将所有选项拼接为一个字符串

In [5]:
f_sel = [re.match(m,data_rev[i]).groups() for i in range(len(data_rev))]

for i in range(len(f_sel)-1):
    if f_sel[i][1] and f_sel[i+1][1] is not None:
        data_rev[i+1] = '%s%s' % (data_rev[i+1],data_rev[i])
        data_rev[i] = 'flag'
        
for i in range(len(data_rev)-1,-1,-1):
    if data_rev[i] == 'flag':
        data_rev.pop(i)
data = list(reversed(data_rev))

问题与答案的处理

In [6]:
que = []
ans = []
for i in range(len(data)):
    if i%2 ==0: que.append(data[i])
    else: ans.append(data[i])
    

ans_split = '([A-F]\.)'
s_ans = [re.split(ans_split,ans[i]) for i in range(len(ans))]


for i in range(len(s_ans)):
    for j in range(len(s_ans[i])):
        s_ans[i][j] = s_ans[i][j].replace('\t',' ')



# 合并答案
for i in range(len(s_ans)):
    for j in range(len(s_ans[i])-1,-1,-2):
        s_ans[i][j-1] = '%s%s' % (s_ans[i][j-1],s_ans[i][j])
        s_ans[i].pop(j)

In [7]:
for i in range(len(s_ans)):
    # 把问题加上
    s_ans[i].reverse()
    s_ans[i].append(que[i])
    s_ans[i].reverse()

In [8]:
import xlwt
book = xlwt.Workbook(encoding='utf-8')
sheet1 = book.add_sheet('test')
for i in range(len(s_ans)):
    for j in range(len(s_ans[i])):
        sheet1.write(i,j,s_ans[i][j])
book.save(r'final_b.xls')